In [1]:
from lightgbm import LGBMClassifier
# pip install bayesian-optimization
# 注意类库名字
from bayes_opt import BayesianOptimization

In [3]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score

def GetNewDataByPandas():
    wine = pd.read_csv("G:/Data/text/wine/wine.csv")
    wine['alcohol**2'] = pow(wine["alcohol"], 2)
    wine['volatileAcidity*alcohol'] = wine["alcohol"] * wine['volatile acidity']
    y = np.array(wine.quality)
    X = np.array(wine.drop("quality", axis=1))
    # X = np.array(wine)

    columns = np.array(wine.columns)

    return X, y, columns


# Read wine quality data from file
X, y, wineNames = GetNewDataByPandas()

# split data to [[0.8,0.2],01]
x_train, x_predict, y_train, y_predict = train_test_split(X, y, test_size=0.10, random_state=100)

In [4]:
from sklearn.datasets import make_classification
# x_train,y_train = make_classification(n_samples=1000,n_features=10,n_classes=2)
def lgb_cv( n_estimators,num_leaves,max_depth,colsample_bytree,subsample,learning_rate,reg_alpha,reg_lambda):
    lcf = LGBMClassifier(n_estimators=int(n_estimators),
                                         num_leaves=int(num_leaves),
                                         max_depth=int(max_depth),
                                         colsample_bytree=colsample_bytree,
                                         subsample=subsample,
                                         learning_rate=learning_rate,
                                         reg_alpha=reg_alpha,
                                         reg_lambda=reg_lambda,
                                         random_state=2018,n_jobs=-1)
    
#     metric = cross_val_score(lcf,x_train,y_train,'roc_auc',cv=5)
    metric = cross_val_score(lcf,x_train, y_train, cv=5, scoring="neg_mean_squared_error")
    print(min(-metric))
#     print('******************************************************************')
    return max(metric)

In [5]:
pbounds = {'n_estimators': (200,800),
                                         'num_leaves':(30,70),
                                         'max_depth': (5,8),
                                         'colsample_bytree': (0.5,0.999),
                                         'subsample': (0.5,0.999),
                                         'learning_rate': (0.001,0.1),
                                         'reg_alpha': (0.1,5),
                                         'reg_lambda': (0.1,5)
                                         }

In [6]:
lgb_bayes = BayesianOptimization(lgb_cv,
                                 pbounds=pbounds,
                                 random_state=1)

In [7]:
lgb_bayes.maximize(
    init_points=30,
    n_iter=300,
)

|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
0.3931034482758621
|  1        | -0.3931   |  0.7081   |  0.07231  |  5.0      |  381.4    |  35.87    |  0.5525   |  1.013    |  0.6724   |
0.42657342657342656
|  2        | -0.4266   |  0.698    |  0.05434  |  6.258    |  611.1    |  38.18    |  4.403    |  0.2342   |  0.8346   |
0.43252595155709345
|  3        | -0.4325   |  0.7082   |  0.05631  |  5.421    |  318.9    |  62.03    |  4.844    |  1.636    |  0.8455   |
0.44982698961937717
|  4        | -0.4498   |  0.9373   |  0.08957  |  5.255    |  223.4    |  36.79    |  4.403    |  0.5819   |  0.7101   |
0.43356643356643354
|  5        | -0.4336   |  0.978    |  0.05378  |  7.076    |  389.3    |  57.46    |  4.19     |  0.1896   |  0.8743   |
0.3931034482758621
|  6        | -0.3931   |  0

0.4
|  37       | -0.4      |  0.9964   |  0.0656   |  7.979    |  676.3    |  69.8     |  0.2365   |  0.6004   |  0.6966   |
0.5051903114186851
|  38       | -0.5052   |  0.999    |  0.001    |  8.0      |  470.7    |  70.0     |  3.562    |  0.1      |  0.5852   |
0.4982698961937716
|  39       | -0.4983   |  0.5      |  0.001    |  8.0      |  236.8    |  70.0     |  0.1      |  0.1      |  0.5      |
0.4
|  40       | -0.4      |  0.7499   |  0.03285  |  7.759    |  510.6    |  31.09    |  0.8088   |  1.019    |  0.5131   |
0.40344827586206894
|  41       | -0.4034   |  0.999    |  0.1      |  5.0      |  200.0    |  30.0     |  0.1      |  0.1      |  0.5      |
0.5294117647058824
|  42       | -0.5294   |  0.8245   |  0.001    |  8.0      |  348.0    |  30.0     |  5.0      |  5.0      |  0.5      |
0.3896551724137931
|  43       | -0.3897   |  0.9799   |  0.1      |  8.0      |  406.6    |  31.87    |  0.1      |  0.1      |  0.5      |
0.4186851211072664
|  44       | -0.4187  

0.5086505190311419
|  75       | -0.5087   |  0.5      |  0.001    |  8.0      |  278.1    |  70.0     |  5.0      |  0.1      |  0.999    |
0.40484429065743943
|  76       | -0.4048   |  0.5      |  0.1      |  8.0      |  363.9    |  41.55    |  0.1      |  0.1      |  0.999    |
0.40559440559440557
|  77       | -0.4056   |  0.5      |  0.1      |  8.0      |  200.0    |  70.0     |  0.1      |  5.0      |  0.999    |
0.4195804195804196
|  78       | -0.4196   |  0.5      |  0.1      |  5.0      |  301.3    |  66.77    |  0.1      |  0.1      |  0.999    |
0.4740484429065744
|  79       | -0.474    |  0.5      |  0.001    |  8.0      |  786.3    |  30.0     |  5.0      |  5.0      |  0.5      |
0.4689655172413793
|  80       | -0.469    |  0.5      |  0.001    |  8.0      |  800.0    |  30.0     |  0.1      |  0.1      |  0.5      |
0.4186851211072664
|  81       | -0.4187   |  0.6209   |  0.1      |  5.0      |  767.5    |  30.73    |  0.1      |  5.0      |  0.999    |
0.429065743

0.4206896551724138
|  113      | -0.4207   |  0.999    |  0.1      |  8.0      |  211.4    |  70.0     |  0.1      |  0.1      |  0.5      |
0.41522491349480967
|  114      | -0.4152   |  0.5      |  0.1      |  8.0      |  328.9    |  30.0     |  0.1      |  0.1      |  0.5      |
0.4125874125874126
|  115      | -0.4126   |  0.999    |  0.01112  |  8.0      |  717.4    |  47.95    |  0.1      |  5.0      |  0.999    |
0.4013840830449827
|  116      | -0.4014   |  0.5776   |  0.1      |  8.0      |  443.7    |  70.0     |  0.1      |  0.1      |  0.5      |
0.4195804195804196
|  117      | -0.4196   |  0.999    |  0.1      |  8.0      |  200.0    |  36.72    |  0.1      |  5.0      |  0.5      |
0.3931034482758621
|  118      | -0.3931   |  0.999    |  0.1      |  5.0      |  362.8    |  30.0     |  0.1      |  0.1      |  0.5      |
0.42657342657342656
|  119      | -0.4266   |  0.999    |  0.007427 |  5.0      |  748.9    |  30.0     |  0.1      |  0.1      |  0.5      |
0.397923875

0.40344827586206894
|  151      | -0.4034   |  0.9975   |  0.07364  |  5.239    |  684.4    |  69.27    |  0.6351   |  4.974    |  0.7961   |
0.39792387543252594
|  152      | -0.3979   |  0.5      |  0.1      |  8.0      |  553.2    |  30.0     |  0.1      |  5.0      |  0.5      |
0.44982698961937717
|  153      | -0.4498   |  0.999    |  0.1      |  5.021    |  800.0    |  41.99    |  5.0      |  0.1474   |  0.999    |
0.3875432525951557
|  154      | -0.3875   |  0.6282   |  0.0717   |  7.789    |  382.1    |  69.67    |  0.11     |  0.6197   |  0.5465   |
0.4160839160839161
|  155      | -0.4161   |  0.5      |  0.1      |  8.0      |  568.3    |  58.83    |  0.1      |  5.0      |  0.999    |
0.4117647058823529
|  156      | -0.4118   |  0.5909   |  0.08314  |  7.998    |  203.3    |  46.07    |  4.572    |  0.1368   |  0.6386   |
0.4117647058823529
|  157      | -0.4118   |  0.8168   |  0.09907  |  7.974    |  411.9    |  30.48    |  3.466    |  4.674    |  0.789    |
0.56747404

0.39100346020761245
|  189      | -0.391    |  0.999    |  0.1      |  8.0      |  359.5    |  58.92    |  0.1      |  0.1      |  0.5      |
0.39100346020761245
|  190      | -0.391    |  0.999    |  0.1      |  8.0      |  352.9    |  70.0     |  0.1      |  0.1      |  0.999    |
0.39792387543252594
|  191      | -0.3979   |  0.5      |  0.1      |  8.0      |  738.8    |  30.0     |  0.1      |  5.0      |  0.5      |
0.4195804195804196
|  192      | -0.4196   |  0.9091   |  0.05773  |  5.423    |  518.1    |  30.25    |  0.9825   |  4.579    |  0.939    |
0.4429065743944637
|  193      | -0.4429   |  0.999    |  0.1      |  5.0      |  453.7    |  42.44    |  0.1      |  5.0      |  0.999    |
0.4463667820069204
|  194      | -0.4464   |  0.999    |  0.1      |  5.0      |  472.5    |  37.36    |  5.0      |  0.1      |  0.999    |
0.4359861591695502
|  195      | -0.436    |  0.999    |  0.1      |  8.0      |  633.5    |  42.7     |  5.0      |  0.1      |  0.5      |
0.40689655

0.39100346020761245
|  227      | -0.391    |  0.999    |  0.1      |  8.0      |  375.8    |  58.44    |  0.1      |  0.1      |  0.999    |
0.44982698961937717
|  228      | -0.4498   |  0.6776   |  0.007199 |  7.661    |  706.8    |  30.15    |  4.858    |  4.204    |  0.5846   |
0.42560553633217996
|  229      | -0.4256   |  0.6044   |  0.1      |  8.0      |  217.3    |  35.77    |  5.0      |  5.0      |  0.5      |
0.39655172413793105
|  230      | -0.3966   |  0.999    |  0.1      |  8.0      |  640.7    |  54.68    |  0.1      |  5.0      |  0.5      |
0.4
|  231      | -0.4      |  0.8333   |  0.09069  |  7.714    |  549.2    |  42.21    |  0.3368   |  4.891    |  0.8715   |
0.41724137931034483
|  232      | -0.4172   |  0.999    |  0.1      |  8.0      |  311.1    |  39.69    |  0.1      |  0.1      |  0.999    |
0.44982698961937717
|  233      | -0.4498   |  0.5      |  0.1      |  5.0      |  297.2    |  30.0     |  5.0      |  0.1      |  0.999    |
0.4359861591695502
|  

0.43006993006993005
|  265      | -0.4301   |  0.7818   |  0.01635  |  5.346    |  494.7    |  60.23    |  0.1263   |  4.723    |  0.5032   |
0.3737024221453287
|  266      | -0.3737   |  0.999    |  0.1      |  8.0      |  774.6    |  58.83    |  0.1      |  0.1      |  0.999    |
0.3896551724137931
|  267      | -0.3897   |  0.855    |  0.02304  |  7.689    |  489.3    |  65.26    |  0.134    |  0.1318   |  0.5501   |
0.4068965517241379
|  268      | -0.4069   |  0.999    |  0.1      |  7.412    |  318.0    |  49.26    |  0.1      |  0.1      |  0.5      |
0.46853146853146854
|  269      | -0.4685   |  0.999    |  0.001    |  8.0      |  772.8    |  30.0     |  0.1      |  0.1      |  0.999    |
0.4359861591695502
|  270      | -0.436    |  0.999    |  0.1      |  8.0      |  787.0    |  42.85    |  5.0      |  0.1      |  0.999    |
0.4241379310344828
|  271      | -0.4241   |  0.5594   |  0.06081  |  7.528    |  378.8    |  43.73    |  0.2307   |  0.1021   |  0.7345   |
0.468531468

0.39655172413793105
|  303      | -0.3966   |  0.999    |  0.1      |  8.0      |  223.5    |  70.0     |  0.1      |  5.0      |  0.999    |
0.4517241379310345
|  304      | -0.4517   |  0.5256   |  0.005435 |  7.095    |  409.4    |  39.43    |  0.1052   |  4.768    |  0.7759   |
0.41379310344827586
|  305      | -0.4138   |  0.6508   |  0.08107  |  7.775    |  647.4    |  45.84    |  0.1528   |  4.913    |  0.5471   |
0.4359861591695502
|  306      | -0.436    |  0.6027   |  0.1      |  8.0      |  414.7    |  55.33    |  4.761    |  1.653    |  0.999    |
0.4020979020979021
|  307      | -0.4021   |  0.5      |  0.1      |  5.0      |  285.0    |  67.81    |  0.1      |  5.0      |  0.999    |
0.4103448275862069
|  308      | -0.4103   |  0.9506   |  0.03554  |  7.845    |  501.5    |  61.99    |  0.3373   |  0.1383   |  0.5281   |
0.4241379310344828
|  309      | -0.4241   |  0.999    |  0.1      |  5.0      |  738.8    |  68.11    |  0.1      |  0.1      |  0.999    |
0.424137931

In [8]:
lgb_bayes.max

{'params': {'colsample_bytree': 0.5,
  'learning_rate': 0.0863809832358118,
  'max_depth': 8.0,
  'n_estimators': 311.37622447823185,
  'num_leaves': 51.24108970302185,
  'reg_alpha': 0.1,
  'reg_lambda': 0.1,
  'subsample': 0.999},
 'target': -0.3689655172413793}

In [34]:
# lgb_bayes.max

{'target': -0.3689655172413793,
 'params': {'colsample_bytree': 0.8583783855472155,
  'learning_rate': 0.031032509411115138,
  'max_depth': 5.064996844968343,
  'n_estimators': 677.1131637191602,
  'num_leaves': 47.10617628689732,
  'reg_alpha': 0.4047738061603211,
  'reg_lambda': 1.179768000968162,
  'subsample': 0.5133383454512672}}

In [9]:
lgb_bayes.maximize(
    init_points=3,
    n_iter=30,
)

|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
0.41522491349480967
|  331      | -0.4152   |  0.6573   |  0.0894   |  6.734    |  310.4    |  61.52    |  3.099    |  0.3642   |  0.7097   |
0.43252595155709345
|  332      | -0.4325   |  0.8389   |  0.09194  |  5.001    |  786.1    |  45.06    |  4.872    |  3.063    |  0.9136   |
0.43252595155709345
|  333      | -0.4325   |  0.7868   |  0.06318  |  5.857    |  552.1    |  60.0     |  4.306    |  3.8      |  0.8483   |
0.40344827586206894
|  334      | -0.4034   |  0.8412   |  0.07095  |  5.13     |  322.9    |  30.04    |  0.5697   |  4.22     |  0.7073   |
0.4206896551724138
|  335      | -0.4207   |  0.928    |  0.0449   |  5.121    |  649.4    |  69.33    |  2.161    |  4.802    |  0.6957   |
0.4125874125874126
|  336      | -0.4126   |  0

In [36]:
lgb_bayes.max

{'target': -0.36551724137931035,
 'params': {'colsample_bytree': 0.856720039465065,
  'learning_rate': 0.036224452081801695,
  'max_depth': 7.739652159606231,
  'n_estimators': 702.8796527935456,
  'num_leaves': 44.622070715099596,
  'reg_alpha': 0.3655896892105577,
  'reg_lambda': 0.2611869320581405,
  'subsample': 0.60946376064088}}